<a href="https://colab.research.google.com/github/Luca-wirthner/2024_Geoinformatique_I/blob/main/surfaces_agricoles_utiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import geopandas as gpd
!pip install pooch
import pooch

In [3]:
# Définition du chemin d'enregistrement du fichier en format ZIP
data_agri = pooch.retrieve(url = 'https://files.swisscarto.ch/geoinf1/vd_dgav_surfaces_agricoles.zip', known_hash = None)

SHA256 hash of downloaded file: 0d866067c647f235d92499e669b8dfb4dfea18da3cfd99434a7e6176a9fba81f
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


In [4]:
import zipfile
import os

In [5]:
# Décompression de l'archive
with zipfile.ZipFile(data_agri, 'r') as zip_ref:
    zip_ref.extractall('/content/surfaces_agricoles')

In [6]:
# Chargement du fichier Geopackage contenant la couche 'agr_dgav_sau'
gpkg_path =  '/content/surfaces_agricoles/vd_dgav_surfaces_agricoles.gpkg'
gdf_agri = gpd.read_file(gpkg_path, layer = 'agr_dgav_sau')

In [1]:
from google.colab import files

# Téléchargement du fichier Geopackage contenant le périmètre numérisé depuis l'ordinateur
uploaded = files.upload()

Saving geoinf-proj-ze-3860.gpkg to geoinf-proj-ze-3860.gpkg


In [7]:
# Lecture du fichier Geopackage téléchargé
file_name = list(uploaded.keys())[0]
# Récupération du nom du fichier Geopackage téléchargé
gdf_perimetre = gpd.read_file(file_name)

/usr/local/lib/python3.10/dist-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'geoinf-proj-ze-3860.gpkg': 'ze_3860' (default), 'geoinf-proj-ze-3860', 'perimetre_evaluation'. Specify layer parameter to avoid this warning.
  result = read_func(


In [13]:
import pandas as pd

# Lecture du fichier CSV contenu dans l'archive
csv_path ='/content/surfaces_agricoles/vd_dgav_sau_utilisation_2023.csv'
df_utilisation_agri = pd.read_csv(csv_path)

In [11]:
# Intersection des polygones des surfaces agricoles avec le périmètre numérisé
gdf_agriculure_intersection = gpd.overlay(gdf_agri, gdf_perimetre, how = 'intersection')

In [26]:
# Jointure sur la colonne 'ident' pour lier les informations contenues dans le fichier CSV
surface_agri_data = gdf_agriculure_intersection.merge(df_utilisation_agri, left_on = 'ident', right_on = 'IDENT', how = 'left')
# Ajout d'une colonne contenant les superficies des surfaces agricoles calculées en hectares (par défaut en mètres)
surface_agri_data['surface_ha'] = surface_agri_data.geometry.area/10000

In [27]:
# Regroupement des surfaces agricoles  par utilisation et sommation de leur superficie
resume_surfaces = surface_agri_data.groupby('UTILISATIO')['surface_ha'].sum().reset_index()
# Sauvegarde du résumé dans un fichier CSV
resume_surfaces.to_csv('/content/resume_surfaces_agricoles.csv', index = False)

In [28]:
# Suppression de la colonne 'IDENT' pur éviter une erreur dans le fichier Geopackage
del(surface_agri_data['IDENT'])
# Sauvegarde dans un fichier Geopackage
surface_agri_data.to_file('/content/surfaces_agricoles_utilisation.gkpg', layer = 'surfaces_agricoles_utilisation', driver = 'GPKG')

/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: File /content/surfaces_agricoles_utilisation.gkpg has GPKG application_id, but non conformant file extension
  ogr_write(


In [29]:
# Téléchargement des fichiers finaux vers l'ordinateur
files.download('/content/resume_surfaces_agricoles.csv')
files.download('/content/surfaces_agricoles_utilisation.gkpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>